**Do a simple test with leaked(supplemental) data,from this test, we can know what to do and what to submit。In this test, I just rank the securities according to their target in a day,not consider the volatility of the daily spread return,So the score will certainly be much lower than other notebooks that use leaked data。The purpose of using leaked data is not to get high scores, but to understand what we need to submit,**

In [ ]:
import pandas as pd

In [ ]:
# use supplemental data 
df = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv', parse_dates=["Date"])

In [ ]:
df = df[:56*2000] # end to 22.2.28

In [ ]:
df['Rank'] = df.groupby("Date")["Target"].rank(ascending=False, method="first") - 1

In [ ]:
df['Rank'] = df['Rank'].astype('int')

In [ ]:
df = df.sort_values(["Date", "Rank"],ascending=True)

In [ ]:
df_submission = df.loc[:,['Date','SecuritiesCode','Rank']]

In [ ]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()
for prices, _, _, _, _, sample_prediction in iter_test:
    df_on_that_day = df_submission[df_submission['Date']==prices["Date"].iloc[0]]
    map_dict = df_on_that_day.set_index("SecuritiesCode")["Rank"]
    sample_prediction["Rank"] = sample_prediction.SecuritiesCode.map(map_dict)
    env.predict(sample_prediction)